<a href="https://colab.research.google.com/github/eileen-b/holmuskdatachallenge/blob/main/Holmusk_Data_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Holmusk Data Challenge
### Jul 2021 

### Problem statement
The goal of the specific task is to compare the real-world efficacy of the two drugs by comparing the risk of events using a method like survival analysis. 

Do keep in mind that as this data comes from EHR and not a clinical trial, the two groups may not have balanced patient characteristics which may confound the results of your analysis. 
You may also have to think about how you would measure and reduce the impact of this confounding in your analysis approach.

# Holmusk Data Challenge
### Jul 2021 

### Problem statement
The goal of the specific task is to compare the real-world efficacy of the two drugs by comparing the risk of events using a method like survival analysis. 

Do keep in mind that as this data comes from EHR and not a clinical trial, the two groups may not have balanced patient characteristics which may confound the results of your analysis. 
You may also have to think about how you would measure and reduce the impact of this confounding in your analysis approach.

## Library

In [1]:
# if error, run the following:
# !pip install lifelines
# !pip install pandas-profiling==2.8.0
# other lib
import os, sys, csv
import pandas as pd
import numpy as np
from numpy import nan
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import lifelines
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
from pandas_profiling import ProfileReport
import plotly.express as px
import seaborn as sns
import sklearn
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler 

## For Google Colab

In [2]:
from google.colab import drive # for google colab
drive.mount('/content/drive')
!ls # read content of file

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive		    Patient_characteristics.csv		sample_data
Event_duration.csv  Patient_characteristics_encode.csv


## Loading of dataset

In [3]:
# loading 'Patient_characteristics.csv' into df and to avoid dropping of the rows from error in csv
pt_df = pd.read_csv('Patient_characteristics.csv', quoting=csv.QUOTE_NONE, error_bad_lines=False)
print(f'From Patient_characteristics.csv')
pt_df

From Patient_characteristics.csv


,patient_id,treatment_variable,sex,age,other_drugs_1,other_drugs_2,other_drugs_3,other_drugs_4,other_drugs_5,other_drugs_6,other_drugs_7,other_drugs_8,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_4,diagnosis_5,diagnosis_6,diagnosis_7,diagnosis_8,diagnosis_9,diagnosis_10,diagnosis_11,diagnosis_12,diagnosis_13,diagnosis_14,diagnosis_15,lab_1,lab_2,lab_3,lab_4,lab_5,lab_6,lab_7,lab_8,Diag_Score_1,Diag_Score_2
0,0,Drug_A,1,86,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,Yes,No,Yes,No,No,Yes,No,No,0.974967,358.572186,NaN,NaN,5.498570,17.060713,1.077035,67.781782,5,5
1,1,Drug_A,2,66,No,No,Yes,No,No,Yes,No,No,No,No,No,No,No,No,Yes,No,No,Yes,No,No,Yes,No,No,0.963403,191.378702,NaN,5.818425,NaN,15.244907,1.207056,NaN,2,3
2,2,Drug_A,2,80,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,No,No,0.826443,NaN,NaN,NaN,5.575702,9.972696,1.262097,96.036930,3,4
3,3,Drug_A,1,83,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0.913825,217.800645,NaN,NaN,NaN,11.430790,1.156128,70.024041,1,0
4,4,Drug_B,2,82,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,0.557880,NaN,145.334844,NaN,5.388846,18.516605,1.543875,41.483204,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19279,19279,Drug_B,2,31,No,No,Yes,No,No,Yes,No,No,Yes,Yes,Yes,No,No,Yes,No,Yes,Yes,No,No,No,No,No,No,2.447895,NaN,NaN,NaN,6.015184,33.482063,1.604321,41.472110,3,6
19280,19280,Drug_A,1,82,No,Yes,Yes,No,No,No,No,Yes,Yes,Yes,No,No,No,Yes,No,No,Yes,No,No,No,No,Yes,No,1.193941,NaN,NaN,NaN,NaN,10.967647,1.026063,NaN,3,3
19281,19281,Drug_A,1,85,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,Yes,0.814004,385.599600,NaN,NaN,5.466813,31.887342,0.992814,220.259124,3,5
19282,19282,Drug_B,2,86,No,No,Yes,No,No,No,Yes,Yes,Yes,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,0.798732,309.703855,NaN,NaN,5.941772,15.489570,1.113782,84.553286,3,3


In [4]:
# load 'Event_duration.csv' into df
event_df = pd.read_csv('Event_duration.csv')
print(f'From Event_duration.csv')
event_df

From Event_duration.csv


,patient_id,"Bleeding_event (1=event, 0=censored)",duration_in_years,treatment_variable
0,0,0,0.019178,Drug_A
1,1,1,0.115068,Drug_A
2,2,0,0.167123,Drug_A
3,3,0,0.002740,Drug_A
4,4,0,0.210959,Drug_B
...,...,...,...,...
19279,19279,0,0.013699,Drug_B
19280,19280,1,1.857534,Drug_A
19281,19281,0,0.024658,Drug_A
19282,19282,0,0.016438,Drug_B


## Data summary and data cleaning

In [10]:
# summary of df
pt_df.describe()

,patient_id,sex,age,lab_1,lab_2,lab_3,lab_4,lab_5,lab_6,lab_7,lab_8,Diag_Score_1,Diag_Score_2
count,25035.000000,25035.000000,25035.000000,25035.000000,8657.000000,1502.000000,2958.000000,13030.000000,24799.000000,20749.000000,17439.000000,25035.000000,25035.000000
mean,9412.815339,1.410226,77.014540,1.082471,361.956546,127.769491,5.992786,6.126911,20.725392,1.425004,110.138801,2.558019,3.498183
std,4967.901206,0.491884,10.965707,0.913624,129.559157,46.572455,1.080341,1.033610,14.634019,0.693667,71.630792,1.520104,1.679557
min,0.000000,1.000000,19.000000,0.099026,60.560746,58.372223,3.672585,3.707502,0.787500,0.755203,6.856340,0.000000,0.000000
25%,6014.500000,1.000000,71.000000,0.708522,275.334267,99.505235,5.372802,5.538908,15.343563,1.059285,67.482910,1.000000,2.000000
50%,9144.000000,1.000000,79.000000,0.882476,335.005549,116.288183,5.761631,5.867453,19.037921,1.203154,91.837229,2.000000,3.000000
75%,13024.500000,2.000000,85.000000,1.130389,419.530932,141.315972,6.269465,6.387519,23.385520,1.594048,130.240451,4.000000,5.000000
max,19283.000000,2.000000,103.000000,15.822263,1198.426526,526.152387,15.437457,19.957902,361.509528,36.271124,2113.760316,6.000000,8.000000


In [5]:
# summary of df
event_df.describe()

,patient_id,"Bleeding_event (1=event, 0=censored)",duration_in_years
count,19284.000000,19284.000000,19284.000000
mean,9641.500000,0.185491,0.355320
std,5566.955631,0.388705,0.566624
min,0.000000,0.000000,0.000000
25%,4820.750000,0.000000,0.032877
50%,9641.500000,0.000000,0.095890
75%,14462.250000,0.000000,0.347945
max,19283.000000,1.000000,2.000000


In [6]:
 # looking at the distribution of both treatment
 event_df['treatment_variable'].value_counts()

Drug_B    10766
Drug_A     8518
Name: treatment_variable, dtype: int64

In [7]:
# dtypes of pt_df var
pt_df.dtypes

patient_id              int64
treatment_variable     object
sex                     int64
age                     int64
other_drugs_1          object
other_drugs_2          object
other_drugs_3          object
other_drugs_4          object
other_drugs_5          object
other_drugs_6          object
other_drugs_7          object
other_drugs_8          object
diagnosis_1            object
diagnosis_2            object
diagnosis_3            object
diagnosis_4            object
diagnosis_5            object
diagnosis_6            object
diagnosis_7            object
diagnosis_8            object
diagnosis_9            object
diagnosis_10           object
diagnosis_11           object
diagnosis_12           object
diagnosis_13           object
diagnosis_14           object
diagnosis_15           object
lab_1                 float64
lab_2                 float64
lab_3                 float64
lab_4                 float64
lab_5                 float64
lab_6                 float64
lab_7     

In [8]:
# dtypes of pt_df var
event_df.dtypes

patient_id                                int64
Bleeding_event (1=event, 0=censored)      int64
duration_in_years                       float64
treatment_variable                       object
dtype: object

In [9]:
# selecting only int var
pt_int = pt_df.select_dtypes(include=['number'])
pt_int

,patient_id,sex,age,lab_1,lab_2,lab_3,lab_4,lab_5,lab_6,lab_7,lab_8,Diag_Score_1,Diag_Score_2
0,0,1,86,0.974967,358.572186,NaN,NaN,5.498570,17.060713,1.077035,67.781782,5,5
1,1,2,66,0.963403,191.378702,NaN,5.818425,NaN,15.244907,1.207056,NaN,2,3
2,2,2,80,0.826443,NaN,NaN,NaN,5.575702,9.972696,1.262097,96.036930,3,4
3,3,1,83,0.913825,217.800645,NaN,NaN,NaN,11.430790,1.156128,70.024041,1,0
4,4,2,82,0.557880,NaN,145.334844,NaN,5.388846,18.516605,1.543875,41.483204,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19279,19279,2,31,2.447895,NaN,NaN,NaN,6.015184,33.482063,1.604321,41.472110,3,6
19280,19280,1,82,1.193941,NaN,NaN,NaN,NaN,10.967647,1.026063,NaN,3,3
19281,19281,1,85,0.814004,385.599600,NaN,NaN,5.466813,31.887342,0.992814,220.259124,3,5
19282,19282,2,86,0.798732,309.703855,NaN,NaN,5.941772,15.489570,1.113782,84.553286,3,3


In [10]:
# selecting only int var
event_df.select_dtypes(include=['number'])

,patient_id,"Bleeding_event (1=event, 0=censored)",duration_in_years
0,0,0,0.019178
1,1,1,0.115068
2,2,0,0.167123
3,3,0,0.002740
4,4,0,0.210959
...,...,...,...
19279,19279,0,0.013699
19280,19280,1,1.857534
19281,19281,0,0.024658
19282,19282,0,0.016438


## Missing Data and strategy

In [14]:
# How much of your data is missing?
print('Looking for missing data from both df')
print('--------------------------------------------------------')
print('From pt_df:')
print('--------------------------------------------------------')
print(pt_df.isnull().sum().sort_values(ascending=False).head())
print('--------------------------------------------------------')
print('From event_df:')
print('--------------------------------------------------------')
print(event_df.isnull().sum().sort_values(ascending=False).head())
print('--------------------------------------------------------')

Looking for missing data from both df
--------------------------------------------------------
From pt_df:
--------------------------------------------------------
lab_3    18133
lab_4    16985
lab_2    12633
lab_5     9257
lab_8     5834
dtype: int64
--------------------------------------------------------
From event_df:
--------------------------------------------------------
treatment_variable                      0
duration_in_years                       0
Bleeding_event (1=event, 0=censored)    0
patient_id                              0
dtype: int64
--------------------------------------------------------


As the missing data exist in many column and rows, it would be unwise to drop these variables due to missing data which have many possible reasons for the data to be missing. 

Such as:
- Patient did not take the lab test
- Patient do not need the test
- Healthcare professional did not ordered the test as there were past records which by assessment is adequate for patient's current primay care
- etc.,

Given that the number of missing variables are not the same across different lab_x values, the missing value are not due to truncation of data or selection bias, it could be that the inclusion criteria extends only to some of the variables in this study (e.g., lab_1), while other variables are optional, where data is available within the the study period.

Within the limits of this dataset, it is hard to fully understand what could be the main reason for the missing data, but could likely to be missing at random instead of selection bias. Among the various lab_x values, there are also outliers detected.

As such the strategy for the missing data in this data challenge would be to proceed with imputation of missing data with median values.

In [16]:
# after imputation
print('Looking for missing data from both df')
print('--------------------------------------------------------')
print('From pt_df:')
print('--------------------------------------------------------')
print(pt_df.isnull().sum().sort_values(ascending=False).head())
print('--------------------------------------------------------')

Looking for missing data from both df
--------------------------------------------------------
From pt_df:
--------------------------------------------------------
Diag_Score_2    0
diagnosis_6     0
diagnosis_4     0
diagnosis_3     0
diagnosis_2     0
dtype: int64
--------------------------------------------------------


### Imputation

In [15]:
# imputation of NaN value with median
# cols with int values only
pt_int_col = pt_int.columns.to_list()
X = pt_df[pt_int_col]
# imputing
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(X)
X = pd.DataFrame(data=imp.transform(X) , columns=X.columns)
# update dataframe to replace NaN values
pt_df.update(X)
# look at output
pt_df

,patient_id,treatment_variable,sex,age,other_drugs_1,other_drugs_2,other_drugs_3,other_drugs_4,other_drugs_5,other_drugs_6,other_drugs_7,other_drugs_8,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_4,diagnosis_5,diagnosis_6,diagnosis_7,diagnosis_8,diagnosis_9,diagnosis_10,diagnosis_11,diagnosis_12,diagnosis_13,diagnosis_14,diagnosis_15,lab_1,lab_2,lab_3,lab_4,lab_5,lab_6,lab_7,lab_8,Diag_Score_1,Diag_Score_2
0,0.0,Drug_A,1.0,86.0,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,Yes,No,Yes,No,No,Yes,No,No,0.974967,358.572186,115.659860,5.759017,5.498570,17.060713,1.077035,67.781782,5.0,5.0
1,1.0,Drug_A,2.0,66.0,No,No,Yes,No,No,Yes,No,No,No,No,No,No,No,No,Yes,No,No,Yes,No,No,Yes,No,No,0.963403,191.378702,115.659860,5.818425,5.867960,15.244907,1.207056,91.960108,2.0,3.0
2,2.0,Drug_A,2.0,80.0,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,Yes,Yes,Yes,Yes,No,No,No,No,No,0.826443,334.494851,115.659860,5.759017,5.575702,9.972696,1.262097,96.036930,3.0,4.0
3,3.0,Drug_A,1.0,83.0,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,0.913825,217.800645,115.659860,5.759017,5.867960,11.430790,1.156128,70.024041,1.0,0.0
4,4.0,Drug_B,2.0,82.0,No,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,No,0.557880,334.494851,145.334844,5.759017,5.388846,18.516605,1.543875,41.483204,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19279,19279.0,Drug_B,2.0,31.0,No,No,Yes,No,No,Yes,No,No,Yes,Yes,Yes,No,No,Yes,No,Yes,Yes,No,No,No,No,No,No,2.447895,334.494851,115.659860,5.759017,6.015184,33.482063,1.604321,41.472110,3.0,6.0
19280,19280.0,Drug_A,1.0,82.0,No,Yes,Yes,No,No,No,No,Yes,Yes,Yes,No,No,No,Yes,No,No,Yes,No,No,No,No,Yes,No,1.193941,334.494851,115.659860,5.759017,5.867960,10.967647,1.026063,91.960108,3.0,3.0
19281,19281.0,Drug_A,1.0,85.0,No,No,Yes,Yes,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,No,Yes,0.814004,385.599600,115.659860,5.759017,5.466813,31.887342,0.992814,220.259124,3.0,5.0
19282,19282.0,Drug_B,2.0,86.0,No,No,Yes,No,No,No,Yes,Yes,Yes,No,No,No,No,No,No,Yes,No,No,No,No,No,No,No,0.798732,309.703855,115.659860,5.759017,5.941772,15.489570,1.113782,84.553286,3.0,3.0


## Exploratory Data Analysis